In [17]:
from matplotlib.collections import LineCollection
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib widget
from typing import Tuple, List, Optional, Dict
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from os.path import abspath, dirname, join
import matplotlib.tri as mtri

from mpl_toolkits.mplot3d import Axes3D
# from ipywidgets import widgets


def _load_files(data_dirpath: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Loads ".csv" files from the specified subfolder of the Resources in the form of a numpy array. 
    
    Args:
        data_dirpath (str): The path where the data to be loaded is found
    Returns: 
        data_array (np.ndarray): An array of shape <bun samples x num. time samples x num attributes>. In this case,
                                      the attributes are: gain A, gain B, velocity, acceleration.
        labels (np.ndarray): A list of sorted integers containing the ids of swipe samples in the array above.
    """
    print("Reading csv volume controller swipe files...")
    print(f"Data directory: {data_dirpath}\n")
    # all_rsc_path = join(os.getcwd(), "Resources")
    # data_dirpath = os.path.abspath(os.path.join(all_rsc_path, specific_path))

    data_ls = []
    swipe_ids = []
    for filename in os.listdir(data_dirpath):
        if filename.endswith(".csv"):
            # split the filename into the name part and the extension part
            name, extension = os.path.splitext(filename)
            filepath = os.path.join(data_dirpath, filename)
            filedata = _get_file_data(filepath)
            data_ls.append(filedata)
    
            # extract swipe sample number from the filename
            swipe_num_str = name[name.find("0"): name.find("_velocity")]
            swipe_ids.append(int(swipe_num_str))
            
    data_array = np.stack(data_ls, axis=0)
    print(f"\nData array shape: {data_array.shape}\n")
    
    # sort labels and the corresponding data array
    swipe_ids = np.array(swipe_ids)
    idx = swipe_ids.argsort()
    data_array = data_array[idx]
    swipe_ids = swipe_ids[idx]
    
    return data_array, swipe_ids

def _get_file_data(filepath: str) -> np.ndarray:
    """
    Obtain the data in the give file, potentially filtering out some rows and columns.

    Args:
        filepath (str): the path to the file whose data is to be extracted
    Returns:
        file_lines (np.ndarray): the extracted data from the given file
    """
    with open(filepath, encoding='ascii') as f:
        # get the data in each file by first stripping and splitting the lines and
        # then creating a numpy array out of these values
        file_lines = []
        print("Filename: ", filepath)
        for line in f:
            line = line.strip(' \t\n\r')
            line = re.split('\t|,', line)
            file_lines.append(line)
        file_lines = np.asarray(file_lines)

        # keep info only from the relevant columns and rows - in this case get everything
        file_lines = (file_lines[:]).astype(np.float32)
        return file_lines

plt.close('all')
specific_path = "Resources/volume_slider_swipes/p001"
data_array, labels = _load_files (specific_path)

Reading csv volume controller swipe files...
Data directory: Resources/volume_slider_swipes/p001

Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe000_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe001_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe002_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe003_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe004_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe005_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe006_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe007_velocity_acceleration_data.csv
Filename:  Resources/volume_slider_swipes/p001\volumeslider_swipe008_velocity_acceleration_data.csv
Fi

In [18]:
def define_plot_env(fig, title: str, ax_labels: List[str]) -> Axes3D: 
    """
    Creates the axis on which to plot. 
    
    Args:
        fig: The figure object that will contain this axis
        title: The title of the plot
        ax_labels: The labels of each of the three axis 

    Returns:
        ax: The axis on which to plot.

    """
    sns.set(context='notebook', style='darkgrid', palette='pastel', font='sans-serif', font_scale=1,
            color_codes=True, rc=None)
    ax = fig.add_subplot(111, projection='3d')

    plt.title(title, fontsize=16)
    sns.axes_style("white")
    
    # spacing of numbers on each axis
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))
    ax.zaxis.set_major_locator(plt.MaxNLocator(5))
    
    # change the fontsize of ticks
    ax.tick_params(axis='x', labelsize=16) 
    ax.tick_params(axis='y', labelsize=16)
    ax.tick_params(axis='z', labelsize=16)
    
    # add labels to each axis
    ax.set_xlabel(ax_labels[0], fontsize=18, labelpad=22)
    ax.set_ylabel(ax_labels[1], fontsize=18, labelpad=22)
    ax.set_zlabel(ax_labels[2], fontsize=18, labelpad=22)
    
    return ax


In [3]:
def plot_gains_time(all_data: np.ndarray, labels: np.ndarray, filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    # prepare axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory over Time"
    ax_labels = ["Time", "Gain A", "Gain B"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
        
    x = np.arange(0, all_data.shape[1]) 
    for i in idx:
        sample = all_data[i, :, :]

        y = sample[:, 0] # gain A
        z = sample[:, 1] # gain B
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_time = [5, 8, 11, 18, 25] # change this for samples to plot, or set to None to plot all samples
filter_gains_time = None
plot_gains_time(data_array, labels, filter_gains_time)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def plot_gains_vel(all_data: np.ndarray, labels: np.ndarray, filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values with the related particle velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory vs Velocity"
    ax_labels = ["Gain A", "Gain B", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    for i in idx: 
        sample = all_data[i, :, :]
        x = sample[:, 0]
        y = sample[:, 1]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_vel = [25]
# filter_gains_vel = None
# plot_gains_vel(data_array, labels, filter_gains_vel)




In [5]:
def plot_gains_acc(all_data: np.ndarray, labels: np.ndarray, filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values with the related particle acceleration for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
      
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory vs Acceleration"
    ax_labels = ["Gain A", "Gain B", "Acceleration"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    for i in idx: 
        sample = all_data[i, :, :]
        x = sample[:, 0]
        y = sample[:, 1]
        z = sample[:, 3]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_acc = [5, 8, 11, 17, 25]
# filter_gains_acc = None
# plot_gains_acc(data_array, labels, filter_gains_acc)


In [6]:
def plot_time_gainA_vel(all_data: np.ndarray, labels: np.ndarray, filter: Optional[List[int]] = None) -> None:
    """
    Plots the values of gain A and the particle motion velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain A Value and Velocity over Time"
    ax_labels = ["Time", "Gain A", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    x = np.arange(0, all_data.shape[1])
    for i in idx: 
        sample = all_data[i, :, :]
        y = sample[:, 0]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_acc = [5, 8, 11, 17, 25]
# filter_time_gainA_vel = None
# plot_time_gainA_vel(data_array, labels, filter_time_gainA_vel)



In [7]:

def plot_time_gainB_vel(all_data: np.ndarray, labels: np.ndarray, filter: Optional[List[int]] = None) -> None:
    """
    Plots the values of gain B and the particle motion velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                               number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain B Value and Velocity over Time"
    ax_labels = ["Time", "Gain B", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    x = np.arange(0, all_data.shape[1])
    for i in idx: 
        sample = all_data[i, :, :]
        y = sample[:, 1]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_time_gainB_vel = [5]
# filter_time_gainB_vel = None
# plot_time_gainB_vel(data_array, labels, filter_time_gainB_vel)


In [8]:
def plot_gains_vel_surface(all_data: np.ndarray, sample_id: int = 0) -> None:
    """
    Plots the gain values with the related particle velocity for one swipe sample.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        sample_id: identity (index in the array) of sample to plot 
    """
      
    assert 0 <= sample_id < all_data.shape[0]
      
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory of Sample " + str(sample_id) + " vs its Velocity"
    ax_labels = ["Gain A", "Gain B", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    sample = all_data[sample_id, :, :]
    x = sample[:, 0]
    y = sample[:, 1]
    z = sample[:, 2]
    
    X, Y = np.meshgrid(x, y)
    triang = mtri.Triangulation(x, y)
    interpolator = mtri.LinearTriInterpolator(triang, z)
    Z = interpolator(X, Y)
        
    ax.plot_surface(X, Y, Z)
    plt.show()

# sample_id = 25
# plot_gains_vel_surface(data_array, sample_id)



In [9]:
def get_condition_dataset(specific_path: str) -> Dict[str, Tuple[np.ndarray, np.ndarray]]:
    condition_dict = {}

    data_dirpath = join("Resources", specific_path)
    for cat_data_container in os.listdir(data_dirpath):
        condition_path = os.path.join(data_dirpath, cat_data_container)
        print(f"\nContainer name: {cat_data_container}")
        data_array, swipe_ids = _load_files(condition_path)
        condition_dict [cat_data_container] = (data_array, swipe_ids)
    return condition_dict


specific_path = "pressure_diff"
condition_dict = get_condition_dataset(specific_path)


Container name: double
Reading csv volume controller swipe files...
Data directory: Resources\pressure_diff\double

Filename:  Resources\pressure_diff\double\double_000.csv
Filename:  Resources\pressure_diff\double\double_001.csv
Filename:  Resources\pressure_diff\double\double_002.csv
Filename:  Resources\pressure_diff\double\double_003.csv
Filename:  Resources\pressure_diff\double\double_004.csv
Filename:  Resources\pressure_diff\double\double_005.csv
Filename:  Resources\pressure_diff\double\double_006.csv
Filename:  Resources\pressure_diff\double\double_007.csv
Filename:  Resources\pressure_diff\double\double_008.csv
Filename:  Resources\pressure_diff\double\double_009.csv
Filename:  Resources\pressure_diff\double\double_010.csv
Filename:  Resources\pressure_diff\double\double_011.csv
Filename:  Resources\pressure_diff\double\double_012.csv
Filename:  Resources\pressure_diff\double\double_013.csv
Filename:  Resources\pressure_diff\double\double_014.csv
Filename:  Resources\pressur

Filename:  Resources\pressure_diff\medium\medium_014.csv
Filename:  Resources\pressure_diff\medium\medium_015.csv
Filename:  Resources\pressure_diff\medium\medium_016.csv
Filename:  Resources\pressure_diff\medium\medium_017.csv
Filename:  Resources\pressure_diff\medium\medium_018.csv
Filename:  Resources\pressure_diff\medium\medium_019.csv
Filename:  Resources\pressure_diff\medium\medium_020.csv
Filename:  Resources\pressure_diff\medium\medium_021.csv
Filename:  Resources\pressure_diff\medium\medium_022.csv
Filename:  Resources\pressure_diff\medium\medium_023.csv
Filename:  Resources\pressure_diff\medium\medium_024.csv
Filename:  Resources\pressure_diff\medium\medium_025.csv
Filename:  Resources\pressure_diff\medium\medium_026.csv
Filename:  Resources\pressure_diff\medium\medium_027.csv
Filename:  Resources\pressure_diff\medium\medium_028.csv
Filename:  Resources\pressure_diff\medium\medium_029.csv

Data array shape: (30, 1000, 2)


Container name: soft
Reading csv volume controller sw

In [10]:
def plot_conditions(condition_dict: Dict[str, Tuple[np.ndarray, np.ndarray]], filter: Optional[List[str]] = None) -> None:
    """
    Plots the gain values over time for several swipe samples.

    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))>
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities
                of swipes to be plotted.
    """
    # prepare axis
    fig = plt.figure(figsize=(16, 12))
    #title = "Swipes under different conditions"
    title = ""
    ax_labels = ["Time", "Gain A", "Gain B"]
    ax = define_plot_env(fig, title, ax_labels)


    # determine number of samples to plot
    if filter is None:
        idx = condition_dict.keys()
    else:
        idx = filter
        for cond in filter:
            assert cond in condition_dict.keys()
    # colors = ['r', 'b', 'g', 'y', 'm', 'p']

    data_sample = next(iter(condition_dict.values()))[0]
    x = np.arange(0, data_sample.shape[1])

    # for cond, data in condition_dict.items():
    for j, cond in enumerate(idx):
        cond_data = condition_dict[cond][0]
        for i in range(0, cond_data.shape[0]):
            sample = cond_data[i, :, :]
            y = sample[:, 0] # gain A
            z = sample[:, 1] # gain B
            ax.plot(x, y, z, color=sns.color_palette()[j], label=cond)
            #ax.plot(x, y, z, color=sns.color_palette()[3], label=cond)
        
    handles, labels = plt.gca().get_legend_handles_labels()
    labels, ids = np.unique(labels, return_index=True)
    handles = [handles[i] for i in ids]
    plt.legend(handles, labels, loc='center left', bbox_to_anchor=(0.08, 0.9), ncol=1, fancybox=True, shadow=True)
    plt.show()

filter_pressure = ["firm"]
plot_conditions(condition_dict, filter_pressure)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
filter_motion = ["firm", "half", "double"]
plot_conditions(condition_dict, filter_motion)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def single_sample_pca(data_arr):
    data_sample = data_arr[-1, :, :]
    print(f"Data sample shape: {data_sample.shape}")
    sc = StandardScaler() # creating a StandardScaler object
    data_sample_std = sc.fit_transform(data_sample) # standardizing the data
    
    # fig0 = plt.figure(figsize=(10, 9))
    # ax0 = fig0.add_subplot(111)
    # ax0.scatter(data_sample)

    pca = PCA()
    data_pca = pca.fit(data_sample_std)

    eig_vect = pca.components_
    print(f"\nEigenvectors: \n{eig_vect}")
    eig_val = pca.explained_variance_
    print(f"\nEigenvalues: \n{eig_val}\n")

    fig = plt.figure(figsize=(10, 9))
    ax = fig.add_subplot(111)

    x = np.arange(0, len(eig_val))
    ax.plot(x, np.cumsum(pca.explained_variance_ratio_))
    ax.set_xlabel('number of components')
    ax.set_ylabel('cumulative explained variance')
    ax.set_title('Components explaining variance')
    plt.show()

    most_important = [np.abs(pca.components_[i]).argmax() for i in range(len(eig_val))]
    print(f"Most Important Columns: {most_important}")

single_sample_pca(data_array)



Data sample shape: (1000, 4)

Eigenvectors: 
[[ 0.7072361   0.7068542   0.00247906  0.0129652 ]
 [ 0.00451846 -0.01996867  0.70687634  0.7070407 ]
 [-0.36537626  0.37437072  0.6091282  -0.59607834]
 [ 0.6052246  -0.5998375   0.35955864 -0.38028377]]

Eigenvalues: 
[1.6482427  1.634163   0.3770406  0.34455645]



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Most Important Columns: [0, 3, 2, 0]
